In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [29]:
url = "https://guide.wisc.edu/undergraduate/letters-science/mathematics/mathematics-ba/mathematics-mathematics-data-risk-analysis-ba/#requirementstext"
page = requests.get(url)

In [30]:
text = BeautifulSoup(page.text, "html.parser")

In [31]:
## Search for Core Math Requirement
#Search for required credits
credit = text.findAll("td", {"class": "hourscol" })
credit
# Search for tags with class 
classtags= text.findAll("td", {"class": "codecol"})
def mod_class(classn):
    str_class = classn.text
    mod_class = str_class.replace(u'\xa0',' ')
    return mod_class
#str_class
## Create dictionary of class lik

In [32]:
for classn in classtags:
    mod_class(classn)

In [34]:
# Get the headings for the previous tables

# First, we find all the tables and only keep the tables containing course info
tables = text.findAll("tbody")

#required core part
table_names = []
#for table in tables[4]:
    #table_names.append(table.find("span", {"class": "odd areasubheader undefined"}))

print(tables[0])

<tbody> <tr class="even areaheader firstrow"><td colspan="2"><span class="courselistcomment areaheader">Core Math Requirement (minimum of six distinct MATH courses for at least 18 credits)</span></td><td class="hourscol"></td></tr> <tr class="odd areasubheader undefined"><td colspan="2"><span class="courselistcomment areasubheader undefined">Linear Algebra</span></td><td class="hourscol">3-5</td></tr> <tr class="even"><td class="codecol"><div class="blockindent" style="margin-left:20px;"><a class="bubblelink code" href="/search/?P=MATH%20320" onclick="return showCourse(this, 'MATH 320');" title="MATH 320">MATH 320</a></div></td><td> Linear Algebra and Differential Equations</td><td class="hourscol"></td></tr> <tr class="orclass even"><td class="codecol orclass"><div class="blockindent" style="margin-left:20px;">or <a class="bubblelink code" href="/search/?P=MATH%20340" onclick="return showCourse(this, 'MATH 340');" title="MATH 340">MATH 340</a></div></td><td colspan="2">  Elementary Ma

In [35]:
# Get the headings for the previous tables

# First, we find all the tables and only keep the tables containing course info
tables = text.findAll("tbody")

table_names = []
for table in tables:
    table_names.append(table.find("span", {"class": "courselistcomment areaheader"}).contents[0])

DS_required = []

for i,table_name in enumerate(table_names):
    ## Append table name
    cat = []
    cat.append({"Name":table_name})
    
    ## Next, get the required credits
    credit = tables[i].findAll("tr")
    
    flag = 0 # marks the start of the category
    categories = []
    for row in credit:
        # Search for rows containing numbers
        if row.find("td",{"class":"hourscol"}) and len(row.find("td",{"class":"hourscol"}).text) > 0:
            credit = row.find("td",{"class":"hourscol"}).text
            if credit == "3-4":
                credit = 1
            else:
                credit = round(int(credit)/3)

            # Start of a section!
            if row.find("a"):
                class_code = row.find("a").contents[0].replace(u'\xa0',' ')
                class_c = (credit,[class_code])
                categories.append(class_c)
            
        else:
            if row.find("a"):
                class_code = row.find("a").contents[0].replace(u'\xa0',' ')
                categories[-1][-1].append(class_code)
                
    need = (cat,categories)
    DS_required.append(need)


ValueError: invalid literal for int() with base 10: '3-5'

[<tbody> <tr class="even areaheader firstrow"><td colspan="2"><span class="courselistcomment areaheader">Core Math Requirement (minimum of six distinct MATH courses for at least 18 credits)</span></td><td class="hourscol"></td></tr> <tr class="odd areasubheader undefined"><td colspan="2"><span class="courselistcomment areasubheader undefined">Linear Algebra</span></td><td class="hourscol">3-5</td></tr> <tr class="even"><td class="codecol"><div class="blockindent" style="margin-left:20px;"><a class="bubblelink code" href="/search/?P=MATH%20320" onclick="return showCourse(this, 'MATH 320');" title="MATH 320">MATH 320</a></div></td><td> Linear Algebra and Differential Equations</td><td class="hourscol"></td></tr> <tr class="orclass even"><td class="codecol orclass"><div class="blockindent" style="margin-left:20px;">or <a class="bubblelink code" href="/search/?P=MATH%20340" onclick="return showCourse(this, 'MATH 340');" title="MATH 340">MATH 340</a></div></td><td colspan="2">  Elementary M

In [78]:
## INFO FROM ELECTIVES

table = text.findAll("tbody")
table_tr = table[0].findAll("tr")

def get_ele(x):
    for ele in x:
        return ele.get_text()
def replace_that(table):
    a = []
    for row in table:
        r = []
        credits = row.findAll("td", {"class": "hourscol" })
        credits = get_ele(credits)
        
        catg = row.findAll("span",{"class":"courselistcomment areasubheader undefined"})
        catg = get_ele(catg)
        classtags= row.findAll("td", {"class": "codecol"})
        if len(classtags) > 0:
            classtags = get_ele(classtags).replace(u'\xa0',' ')
            classtags = classtags.replace(u'\u200b',' ')
        
        r.append(credits)
        r.append(catg)
        r.append(classtags)
        a.append(r)
    return(a[1:])

tb = replace_that(table_tr)
#tb = list(tb)

#Now, we aim to clean up tb


categories = []
for row in tb:
    ## Categories
    if row[0] != None and len(row[0]) > 0:
        ## Categories
        if row[0] in ["0-3","3-5"]:
            row[0] = 1
        elif row[1] == "Select a distinct introduction course or sequence: ":
            category = ({"Name":row[1]},2,['STAT 340','STAT/ M E  424'])
            categories.append(category)
            pass
        elif row[0] in ["0-6","3-6"]:
            row[0] = 2
        elif row[0] == "6-14":
            row[0] = 3
        else:
            row[0] = round(int(row[0])/3)
        category = ({"Name":row[1]},row[0],[])
        categories.append(category)
    elif len(row[2]) > 0:
        categories[-1][-1].append(row[2])

categories = categories[:-1]
categories.pop(7)
categories

[({'Name': 'Linear Algebra'},
  1,
  ['MATH 320', 'or MATH 340', 'or MATH 341', 'or MATH 375']),
 ({'Name': 'Probability (Complete at least one)'},
  1,
  ['MATH/ STAT  431', 'or MATH/ STAT  309', 'MATH 531']),
 ({'Name': 'Statistics'}, 1, ['MATH/ STAT  310', 'or STAT 312']),
 ({'Name': 'Intermediate Mathematics Requirement (complete at least one)'},
  2,
  ['MATH 321& MATH 322', 'MATH 341', 'MATH 375', 'MATH 421']),
 ({'Name': 'Advanced Mathematics Requirement (select one)'},
  1,
  ['MATH/ COMP SCI  514', 'MATH 521', 'MATH 531', 'MATH 535', 'MATH 540']),
 ({'Name': 'Elective to reach required six courses and 18 credits'},
  2,
  ['MATH/ COMP SCI  513',
   'MATH/ COMP SCI  514',
   'MATH 519',
   'MATH 521',
   'MATH 522',
   'MATH/ COMP SCI/ I SY E/ STAT  525',
   'MATH 531',
   'MATH 535',
   'MATH 540',
   'MATH 541',
   'MATH 542',
   'MATH 605',
   'MATH 619',
   'MATH 627',
   'MATH 629',
   'MATH/ I SY E/ OTM/ STAT  632',
   'MATH 635',
   'MATH 319',
   'MATH 321',
   'MATH 32

In [31]:
DS = DS_required + categories
DS

[([{'Name': 'Foundational Math Courses'}],
  [(2, ['MATH 221', 'MATH 217', 'MATH 275']), (1, ['MATH 222', 'MATH 276'])]),
 ([{'Name': 'Foundational Data Science Courses'}],
  [(1, ['STAT 240']),
   (1, ['STAT 340']),
   (1, ['COMP SCI 220', 'COMP SCI 300']),
   (1, ['COMP SCI 320']),
   (1, ['L I S 461'])]),
 ({'Name': 'Machine Learning'},
  1,
  ['COMP SCI/ E C E/ M E  532',
   'COMP SCI/ E C E/ M E  539',
   'COMP SCI 540',
   'MATH 535',
   'STAT 451',
   'STAT 453']),
 ({'Name': 'Advanced Computing'},
  1,
  ['COMP SCI 400',
   'COMP SCI 412',
   'COMP SCI/ STAT  471',
   'COMP SCI/ MATH  513',
   'COMP SCI/ MATH  514',
   'COMP SCI/ E C E/ I SY E  524',
   'COMP SCI 564',
   'GEOG 573',
   'GEOG 574']),
 ({'Name': 'Statistical Modeling'},
  1,
  ['ECON 400',
   'ECON 410',
   'STAT/ MATH  309',
   'STAT/ MATH  310',
   'STAT 311',
   'STAT 312',
   'STAT 349',
   'STAT 351',
   'STAT 421',
   'STAT/ M E  424',
   'STAT/ MATH  431',
   'STAT 443',
   'STAT 456',
   'STAT 461',
   '